# GPS-Enhanced Image Retrieval - exploration

In [1]:
import json
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import BallTree
import h5py
import torch
import os
from PIL import Image
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import pickle
from gps_helpers import cluster_locations, compute_location_centroids, get_cluster_members, compute_gps_distances

Skipping import of cpp extensions due to incompatible torch version 2.9.0+cu126 for torchao version 0.13.0


## Load Data

In [2]:
DATA_PATH = "../data"
with open(f"{DATA_PATH}/database/database_lite.json", "r") as f:
    db_data = json.load(f)
    db_imgs = np.array(db_data["im_paths"])
    db_loc = np.array(db_data["loc"])

with open(f"{DATA_PATH}/query/query_lite.json", "r") as f:
    query_data = json.load(f)
    query_imgs = np.array(query_data["im_paths"])
    query_loc = np.array(query_data["loc"])

with h5py.File(f"{DATA_PATH}/london_lite_gt.h5", "r") as f:
    gt_sim = f["sim"][:].astype(np.uint8)

print(f"Database: {len(db_imgs)} images")
print(f"Query: {len(query_imgs)} images")

Database: 1000 images
Query: 500 images


## Evaluation Functions

In [3]:
def recall(ranks, pidx, ks):
    recall_at_k = np.zeros(len(ks))
    for qidx in range(ranks.shape[0]):
        for i, k in enumerate(ks):
            if np.sum(np.in1d(ranks[qidx, :k], pidx[qidx])) > 0:
                recall_at_k[i:] += 1
                break
    recall_at_k /= ranks.shape[0]
    return recall_at_k


def apk(pidx, rank, k):
    if len(rank) > k:
        rank = rank[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(rank):
        if p in pidx and p not in rank[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(pidx), k)


def mapk(ranks, pidxs, k):
    return np.mean([apk(a, p, k) for a, p in zip(pidxs, ranks)])


def mapk_many(ranks, pidxs, ks):
    return np.array([mapk(ranks, pidxs, k) for k in ks], dtype=float)


def average_precision(relevant, retrieved):
    precisions = []
    rel = 0
    for i in range(len(retrieved)):
        if retrieved[i] in relevant:
            rel += 1
            precisions.append(rel / (i + 1))
    return sum(precisions) / len(relevant) if len(relevant) > 0 else 0


def mean_average_precision(all_relevant, all_retrieved):
    total = 0
    for qid in all_relevant:
        total += average_precision(all_relevant[qid], all_retrieved.get(qid, []))
    return total / len(all_relevant)


def l2_normalize(x, axis=1, eps=1e-12):
    norm = np.linalg.norm(x, axis=axis, keepdims=True)
    return x / (norm + eps)


def get_relevant_images(gt_similarity_matrix, query_idx):
    return np.where(gt_similarity_matrix[query_idx, :] == 1)[0]

## Extract features

In [8]:
MODEL_NAME = "facebook/dinov3-vith16plus-pretrain-lvd1689m"
FEAT_DIM = 1280
POOLING = "GeM"
GEM_P = 3.0
MIN_SAMPLES=8

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# DINOv3 models use DINOv2 processor (compatible)
processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
print(f"Loaded {MODEL_NAME}")

In [ ]:
def extract_features_gem(image_paths, model, processor, device, p=3.0):
    features = np.zeros((len(image_paths), FEAT_DIM), dtype=np.float32)
    for i, img_path in enumerate(tqdm(image_paths)):
        img = Image.open(os.path.join("data/", img_path))
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.inference_mode():
            outputs = model(**inputs)
        gem_feat = outputs.last_hidden_state[:, 1:, :].clamp(min=1e-6).pow(p).mean(dim=1).pow(1.0 / p)[0]
        features[i] = gem_feat.cpu().numpy()
    return l2_normalize(features, axis=1)

In [ ]:
print("Extracting database features...")
db_features = extract_features_gem(db_imgs, model, processor, device, p=GEM_P)

print("Extracting query features...")
query_features = extract_features_gem(query_imgs, model, processor, device, p=GEM_P)

print(f"Database features: {db_features.shape}")
print(f"Query features: {query_features.shape}")

## GPS Clustering -> finding optimal epsilon (distance) and minimum number of samples per location

In [4]:
d = pairwise_distances(db_loc[:, :2])
print(np.median(d[d>0])) # media sitance, interesting for our location grouping

2918.9292724655475


In [13]:
coords = db_loc[:, :2].astype(np.float64)
tree = BallTree(coords, metric='euclidean')

dist2, _ = tree.query(coords, k=2)
nn = dist2[:, 1]

k5 = 5
dist5, _ = tree.query(coords, k=k5+1)
kdist5 = dist5[:, k5]

k8 = 8
dist8, _ = tree.query(coords, k=k8+1)
kdist8 = dist8[:, k8]

k10 = 10
dist10, _ = tree.query(coords, k=k10+1)
kdist10 = dist10[:, k10]

k20 = 20
dist20, _ = tree.query(coords, k=k20+1)
kdist20 = dist20[:, k20]

percentiles = [10, 25, 50, 75, 90]

print("1-NN (m) percentiles:", np.percentile(nn, percentiles))
print(f"5-NN (m) percentiles:", np.percentile(kdist5, percentiles))
print(f"8-NN (m) percentiles:", np.percentile(kdist8, percentiles))
print(f"10-NN (m) percentiles:", np.percentile(kdist10, percentiles))
print(f"20-NN (m) percentiles:", np.percentile(kdist20, percentiles))

1-NN (m) percentiles: [ 1.15798445  2.55923067  4.7774488   7.76391717 11.67857255]
5-NN (m) percentiles: [ 6.3019239   9.92692027 16.62121606 25.64483338 40.59916861]
8-NN (m) percentiles: [11.39513284 15.86251924 27.76659958 43.50936493 63.57259156]
10-NN (m) percentiles: [14.46676847 19.51624367 35.1837066  53.64121166 82.85689082]
20-NN (m) percentiles: [ 33.84342856  45.00901583  79.50986293 120.76087156 321.49932229]


In [ ]:
EPS_ = [10, 20, 30, 40, 50, 75,100] 
MIN_SAMPLES_ = [2,4,8]
best_option_recall1 = 0.0

for EPS in EPS_:
    for MIN_SAMPLES in MIN_SAMPLES_:
        db_clusters = cluster_locations(db_loc, eps=EPS, min_samples=MIN_SAMPLES)

        n_clusters = len(np.unique(db_clusters[db_clusters >= 0]))
        n_noise = np.sum(db_clusters == -1)

        centroids, cluster_members = compute_location_centroids(db_features, db_clusters)

        centroid_matrix = np.zeros((len(centroids), FEAT_DIM), dtype=np.float32)
        cluster_id_to_idx = {}
        idx_to_cluster_id = {}
        for idx, (cluster_id, centroid) in enumerate(centroids.items()):
            centroid_matrix[idx] = centroid
            cluster_id_to_idx[cluster_id] = idx
            idx_to_cluster_id[idx] = cluster_id

        centroid_matrix = l2_normalize(centroid_matrix, axis=1)
        centroid_similarities = cosine_similarity(query_features, centroid_matrix)
        best_clusters = np.argmax(centroid_similarities, axis=1)

        ranks_a = np.zeros((Q, len(db_imgs)), dtype=int)

        for q_idx in range(Q):
            best_cluster_idx = best_clusters[q_idx]
            cluster_id = idx_to_cluster_id[best_cluster_idx]

            cluster_member_indices = np.array(cluster_members[cluster_id])

            cluster_sims = similarities[q_idx, cluster_member_indices]
            sorted_cluster_indices = cluster_member_indices[np.argsort(-cluster_sims)]

            all_other_indices = np.setdiff1d(np.arange(len(db_imgs)), cluster_member_indices)
            other_sims = similarities[q_idx, all_other_indices]
            sorted_other_indices = all_other_indices[np.argsort(-other_sims)]

            ranks_a[q_idx] = np.concatenate([sorted_cluster_indices, sorted_other_indices])

        all_ret_a = {q: ranks_a[q] for q in range(Q)}
        recall_a = recall(ranks_a, pidx, ks)
        mAPs_a = mapk_many(ranks_a, pidx, ks)
        map_a = mean_average_precision(all_rel, all_ret_a)
        
        if recall_a[0] >best_option_recall1:
            best_option_recall1 = recall_a[0]
            best_option = (EPS, MIN_SAMPLES)
            
print(f"best option is {best_option}")

## Exploring if approach B works better when using smaller distances between centroids/more centroids

In [ ]:
EPS_ = [10, 20, 30, 40, 50, 75, 100]
MIN_SAMPLES_ = [2, 4, 8]
ALPHAS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

results = []
best_overall = None
best_recall1 = 0.0

for EPS in EPS_:
    for MIN_SAMPLES in MIN_SAMPLES_:
        db_clusters = cluster_locations(db_loc, eps=EPS, min_samples=MIN_SAMPLES)
        n_clusters = len(np.unique(db_clusters[db_clusters >= 0]))
        n_noise = np.sum(db_clusters == -1)
        print(f"[EPS={EPS}, MIN_SAMPLES={MIN_SAMPLES}] clusters={n_clusters}, noise={n_noise}")

        centroids, cluster_members = compute_location_centroids(db_features, db_clusters)

        centroid_matrix = np.zeros((len(centroids), FEAT_DIM), dtype=np.float32)
        cluster_id_to_idx, idx_to_cluster_id = {}, {}
        for idx, (cluster_id, centroid) in enumerate(centroids.items()):
            centroid_matrix[idx] = centroid
            cluster_id_to_idx[cluster_id] = idx
            idx_to_cluster_id[idx] = cluster_id

        centroid_matrix = l2_normalize(centroid_matrix, axis=1)
        similarities = cosine_similarity(query_features, db_features)
        centroid_similarities = cosine_similarity(query_features, centroid_matrix)

        for alpha in ALPHAS:
            combined_similarities = np.empty_like(similarities)
            for q_idx in range(Q):
                for db_idx in range(len(db_imgs)):
                    cluster_id = db_clusters[db_idx]
                    if cluster_id >= 0:
                        cluster_idx = cluster_id_to_idx[cluster_id]
                        sim_centroid = centroid_similarities[q_idx, cluster_idx]
                    else:
                        sim_centroid = 0
                    sim_image = similarities[q_idx, db_idx]
                    combined_similarities[q_idx, db_idx] = (
                        alpha * sim_centroid + (1 - alpha) * sim_image
                    )

            ranks = np.argsort(-combined_similarities, axis=1)
            all_ret = {q: ranks[q] for q in range(Q)}

            recall_vals = recall(ranks, pidx, ks)
            mAPs = mapk_many(ranks, pidx, ks)
            MAP = mean_average_precision(all_rel, all_ret)

            result = {
                "EPS": EPS,
                "MIN_SAMPLES": MIN_SAMPLES,
                "alpha": alpha,
                "MAP": MAP,
                "Recall@1": recall_vals[0],
                "Recall@5": recall_vals[1],
                "Recall@10": recall_vals[2],
                "Recall@20": recall_vals[3],
            }
            results.append(result)

            if recall_vals[0] > best_recall1:
                best_recall1 = recall_vals[0]
                best_overall = result

print("\n" + "="*60)
print("BEST CONFIGURATION (by Recall@1)")
print("="*60)
print(best_overall)
